In [1]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers import Convolution2D, MaxPooling2D
from keras import backend as K

In [2]:
K.clear_session()
data_train='/Users/JuanVillalba/Desktop/ImageProcess/CarneDataset/train'
data_test='/Users/JuanVillalba/Desktop/ImageProcess/CarneDataset/test'

In [3]:
epocas=20 #numero de veces para el entrenamiento
altura, longitud=100, 100 #tamano de pixeles
batch_size=32 #número de imágenes que vamos a procesar durante el entrenamiento
pasos=1000 #número de veces que se va a procesar la información durante las épocas
pasos_validacion=200 # cuantas veces se van a ejecutar los pasos de validación
filtrosConv1=32 #profundidad de 32 
filtrosConv2=64 #profundidad de 64
tamano_filtro1=[3,3]
tamano_filtro2=[2,2]
tamano_pool=[2,2]
clases=8 #cantidad de carpetas para la clasificación
lr=0.005 #Ajustes de la red neuronal para acercar a una óptima revisión

In [4]:
#Preprocesamiento de imágenes

train_datagen=ImageDataGenerator(
rescale=1./255, #Valores pixeles de la imagen para mejorar el entrenamiento
shear_range=0.3, #Generar imágenes inclinada
zoom_range=0.3, #Algunas hará zoom
horizontal_flip=True #Para distinguir direccionalidad de la imagen
)


In [5]:
test_datagen=ImageDataGenerator(
	rescale=1./255 
)

In [6]:
#Procesa todas las imágenes de todas las carpetas
image_train = train_datagen.flow_from_directory(
	data_train,
	target_size=(altura, longitud),
	batch_size=batch_size,
	class_mode='categorical'
)

Found 1633 images belonging to 8 classes.


In [7]:
image_test=test_datagen.flow_from_directory(
	data_test,
	target_size=(altura, longitud),
	batch_size=batch_size,
	class_mode='categorical'
)

Found 810 images belonging to 8 classes.


In [8]:
#Crear la red CNN
cnn=Sequential()
cnn.add(Convolution2D(filtrosConv1, 
	tamano_filtro1, 
	padding='same', 
	input_shape=(altura,longitud,3), 
	activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Convolution2D(filtrosConv2, 
	tamano_filtro1, padding='same', 
	input_shape=(altura,longitud,3), 
	activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

2022-09-04 11:22:42.922563: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 100, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 50, 50, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 50, 50, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 25, 25, 64)       0         
 2D)                                                             
                                                                 
Total params: 19,392
Trainable params: 19,392
Non-trainable params: 0
_________________________________________________________________


In [10]:
#Aplanar información
cnn.add(Flatten())
cnn.add(Dense(256,activation='relu')) #Añade una capa más de 256 neuronas en activación relu que es la unidad lineal rectificada.
cnn.add(Dropout(0.5))#Durante el entrenamiento se apagará el 50% de las neuronas en cada paso de las 256, caminos alternos para revisar la data.
cnn.add(Dense(clases, activation='softmax')) #El valor de porcentaje más alto de identificación de clase clasificará la imagen
cnn.compile(loss='categorical_crossentropy', 
            optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
            metrics=['accuracy'])

In [11]:
cnn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 100, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 50, 50, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 50, 50, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 25, 25, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 40000)             0         
                                                                 
 dense (Dense)               (None, 256)               1

In [12]:
history=cnn.fit_generator(image_train, 
        steps_per_epoch=pasos, 
        epochs=epocas, 
        validation_data=image_test, 
        validation_steps=pasos_validacion)

/var/folders/75/z2s0x66143n_008wgn0h_z9h0000gp/T/ipykernel_7729/188802715.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=cnn.fit_generator(image_train,


Epoch 1/20
1000/1000 [==============================] - 19s 18ms/step - loss: 2.6521 - accuracy: 0.5487 - val_loss: 1.4409 - val_accuracy: 0.5667


In [13]:
dir='/Users/JuanVillalba/Desktop/ImageProcess/modelo/'

if not os.path.exists(dir):
	os.mkdir(dir)
cnn.save('/Users/JuanVillalba/Desktop/ImageProcess/modelo.h5')
cnn.save_weights('/Users/JuanVillalba/Desktop/ImageProcess//modelo/pesos.h5')